In [11]:
import importlib
import subprocess
import sys
import os

packages = ['fsspec','lxml','gsw','matplotlib','netCDF4', 'numpy', 'pandas','scipy', 'xarray','imageio']  

for package in packages:
    if importlib.util.find_spec(package) is None:
        subprocess.check_call([sys.executable, '-m' ,'pip','install',package])

In [12]:
#attempt using process_kdata_optaa and profilers.py
%matplotlib inline

import csv
import io
import numpy as np
import os
import pandas as pd
import re
import sys
import time
import xarray as xr #needed "conda install -c conda-forge xarray" in terminal

from ooi_data_explorations.common import get_vocabulary, load_kdata, N_CORES, FILL_INT
from ooi_data_explorations.uncabled.process_optaa import optaa_cspp
from ooi_data_explorations.uncabled.process_spkir import spkir_cspp
from ooi_data_explorations.profilers import create_profile_id, bin_profiles

from concurrent.futures import ProcessPoolExecutor
from functools import partial
from matplotlib import pyplot as plt
from tqdm import tqdm


from matplotlib import pyplot as plt

In [13]:
# OPTAA request params
site = 'CE02SHSP'           # OOI Net site designator
node = 'SP001'              # OOI Net node designator
sensor = '04-OPTAAJ000'     # OOI Net sensor designator
stream = 'optaa_dj_cspp_instrument_recovered'  # OOI Net stream name
method = 'recovered_cspp'   # OOI Net data delivery method
level = 'profiler'              # local directory name, level below site
instrmt = 'optaa'           # local directory name, instrument below level

# We are after the the data from deployment 13
vocab = get_vocabulary(site, node, sensor)[0]
deploy = 13

# load data from the local kdata directory
optaa = load_kdata(site, node, sensor, method, stream, ('*deployment%04d*OPTAA*.nc' % deploy))

# set up the calibration file path and name
cal_path = os.path.join(os.path.expanduser('~'), 'acs_workshop/data/reprocess', site.lower(), level, instrmt)
cal_path = os.path.abspath(cal_path)
if not os.path.exists(cal_path):
    os.makedirs(cal_path)

cal_file_optaa = ('{}.{}.{}.deploy{:02d}.cal_coeffs.json'.format(site.lower(), level, instrmt, deploy))
cal_file_optaa = os.path.join(cal_path, cal_file_optaa)

Loading and Processing Data Files: 100%|██████████| 1/1 [00:01<00:00,  1.70s/it]


In [14]:
# SPKIR request params
site = 'CE02SHSP'           # OOI Net site designator
node = 'SP001'              # OOI Net node designator
sensor = '06-SPKIRJ000'     # OOI Net sensor designator
stream = 'spkir_abj_cspp_instrument_recovered'  # OOI Net stream name
method = 'recovered_cspp'   # OOI Net data delivery method
level = 'profiler'              # local directory name, level below site
instrmt = 'spkir'           # local directory name, instrument below level

# We are after the the data from deployment 13
vocab = get_vocabulary(site, node, sensor)[0]
deploy = 13

# load data from the local kdata directory
spkir = load_kdata(site, node, sensor, method, stream, ('*deployment%04d*SPKIR*.nc' % deploy))

# set up the calibration file path and name
cal_path = os.path.join(os.path.expanduser('~'), 'acs_workshop/data/reprocess', site.lower(), level, instrmt)
cal_path = os.path.abspath(cal_path)
if not os.path.exists(cal_path):
    os.makedirs(cal_path)

cal_file_spkir = ('{}.{}.{}.deploy{:02d}.cal_coeffs.json'.format(site.lower(), level, instrmt, deploy))
cal_file_spkir = os.path.join(cal_path, cal_file_spkir)

Loading and Processing Data Files: 100%|██████████| 1/1 [00:00<00:00,  2.77it/s]


In [6]:
print(len(optaa),"variables")
print(len(optaa.time),"entries")

27 variables
131979 entries


In [7]:
print(len(spkir),"variables")
print(len(spkir.time),"entries")

18 variables
260017 entries


In [8]:
optaa = optaa_cspp(optaa, cal_file_optaa)

Creating and adding a profile variable to the data set ...
Smoothing and binning each profile into 25 cm depth bins: 100%|██████████| 170/170 [00:16<00:00, 10.25it/s]


Converting attenuation data ...: 100%|██████████| 40452/40452 [00:07<00:00, 5251.47it/s]


In [8]:
#spkir = spkir_cspp(spkir, cal_file_spkir)

In [9]:
print(len(optaa),"variables")
print(len(optaa.time),"entries")

38 variables
40452 entries


In [10]:
spkir = spkir_cspp(spkir)

In [11]:
#choosing only high density data region
#spkir = spkir.sel(time=slice('2019-06-25T00:00:00',None))
optaa = optaa.sel(time=slice('2019-06-25T00:00:00',None))

In [12]:
print(len(optaa),'variables')

38 variables


In [13]:
optaa

<xarray.Dataset>
Dimensions:                       (time: 38972, wavelength_number: 100)
Coordinates:
  * time                          (time) datetime64[ns] 2019-06-25T06:44:40.6...
  * wavelength_number             (wavelength_number) int64 0 1 2 3 ... 97 98 99
Data variables: (12/38)
    external_temp_raw             (time) int64 41173 41173 41172 ... 39121 39017
    deployment                    (time) int64 13 13 13 13 13 ... 13 13 13 13 13
    lat                           (time) float64 44.63 44.63 ... 44.63 44.63
    sea_water_pressure            (time) float64 72.5 72.39 ... 1.261 1.053
    lon                           (time) float64 -123.7 -123.7 ... -123.7 -123.7
    internal_temp_raw             (time) int64 51495 51493 51494 ... 50861 50832
    ...                            ...
    wavelength_c                  (time, wavelength_number) float64 399.4 ......
    c_signal                      (time, wavelength_number) int64 543 ... -99...
    c_reference                   (time, wavelength_number) int64 618 ... -99...
    beam_attenuation              (time, wavelength_number) float64 nan ... nan
    cpg                           (time, wavelength_number) float64 1.458 ......
    cpg_ts                        (time, wavelength_number) float64 nan ... nan
Attributes: (12/67)
    node:                               SP001
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_cspp
    stream:                             optaa_dj_cspp_instrument_recovered
    ...                                 ...
    geospatial_lon_max:                 -123.69743
    geospatial_lon_units:               degrees_east
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down

In [14]:
bdt = optaa.time.min()
edt = optaa.time.max()

In [15]:
optaa2 = optaa.assign_coords({'profile': np.unique(optaa.profile)})

In [16]:
optaa2

<xarray.Dataset>
Dimensions:                       (time: 38972, profile: 159,
                                   wavelength_number: 100)
Coordinates:
  * profile                       (profile) int64 7 8 9 10 ... 166 168 169 170
  * time                          (time) datetime64[ns] 2019-06-25T06:44:40.6...
  * wavelength_number             (wavelength_number) int64 0 1 2 3 ... 97 98 99
Data variables: (12/37)
    external_temp_raw             (time) int64 41173 41173 41172 ... 39121 39017
    deployment                    (time) int64 13 13 13 13 13 ... 13 13 13 13 13
    lat                           (time) float64 44.63 44.63 ... 44.63 44.63
    sea_water_pressure            (time) float64 72.5 72.39 ... 1.261 1.053
    lon                           (time) float64 -123.7 -123.7 ... -123.7 -123.7
    internal_temp_raw             (time) int64 51495 51493 51494 ... 50861 50832
    ...                            ...
    wavelength_c                  (time, wavelength_number) float64 399.4 ......
    c_signal                      (time, wavelength_number) int64 543 ... -99...
    c_reference                   (time, wavelength_number) int64 618 ... -99...
    beam_attenuation              (time, wavelength_number) float64 nan ... nan
    cpg                           (time, wavelength_number) float64 1.458 ......
    cpg_ts                        (time, wavelength_number) float64 nan ... nan
Attributes: (12/67)
    node:                               SP001
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_cspp
    stream:                             optaa_dj_cspp_instrument_recovered
    ...                                 ...
    geospatial_lon_max:                 -123.69743
    geospatial_lon_units:               degrees_east
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down

In [17]:
optaa

<xarray.Dataset>
Dimensions:                       (time: 38972, wavelength_number: 100)
Coordinates:
  * time                          (time) datetime64[ns] 2019-06-25T06:44:40.6...
  * wavelength_number             (wavelength_number) int64 0 1 2 3 ... 97 98 99
Data variables: (12/38)
    external_temp_raw             (time) int64 41173 41173 41172 ... 39121 39017
    deployment                    (time) int64 13 13 13 13 13 ... 13 13 13 13 13
    lat                           (time) float64 44.63 44.63 ... 44.63 44.63
    sea_water_pressure            (time) float64 72.5 72.39 ... 1.261 1.053
    lon                           (time) float64 -123.7 -123.7 ... -123.7 -123.7
    internal_temp_raw             (time) int64 51495 51493 51494 ... 50861 50832
    ...                            ...
    wavelength_c                  (time, wavelength_number) float64 399.4 ......
    c_signal                      (time, wavelength_number) int64 543 ... -99...
    c_reference                   (time, wavelength_number) int64 618 ... -99...
    beam_attenuation              (time, wavelength_number) float64 nan ... nan
    cpg                           (time, wavelength_number) float64 1.458 ......
    cpg_ts                        (time, wavelength_number) float64 nan ... nan
Attributes: (12/67)
    node:                               SP001
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_cspp
    stream:                             optaa_dj_cspp_instrument_recovered
    ...                                 ...
    geospatial_lon_max:                 -123.69743
    geospatial_lon_units:               degrees_east
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down

In [18]:
#profile = optaa.sel(profile = 150)
profiles = optaa.groupby('profile')
profile = profiles[130]

In [ ]:
depths = optaa.groupby('depth')
depth = depths[5]

In [ ]:
#profile = profile.assign_coords({'wavelength_a':np.unique(profile.wavelength_a.values)})

In [ ]:

XA = np.array(profile.wavelength_number.values)
YA = np.array(profile.depth.values).astype(np.float64)
XA, YA = np.meshgrid(XA, YA)
ZA = np.array(profile.apg_ts_s.values)

In [ ]:
XC = np.array(profile.wavelength_number.values)
YC = np.array(profile.depth.values).astype(np.float64)
XC, YC = np.meshgrid(XC, YC)
ZC = np.array(profile.cpg_ts.values)

In [ ]:
pnum=130
profile = profiles[pnum]
XA = np.array(profile.wavelength_number.values)
YA = np.array(profile.depth.values).astype(np.float64)
XA, YA = np.meshgrid(XA, YA)
ZA = np.array(profile.apg_ts_s.values)
XC = np.array(profile.wavelength_number.values)
YC = np.array(profile.depth.values).astype(np.float64)
XC, YC = np.meshgrid(XC, YC)
ZC = np.array(profile.cpg_ts.values)

fig = plt.figure(figsize = (11,5.5))
grid = (1,1)
ax1 = plt.subplot2grid(grid, (0,0), colspan = 1, rowspan = 1, projection = '3d')

surf1 = ax1.plot_surface(YA, XA, ZA, cmap = 'jet', linewidth = 0, antialiased = True)

fig.colorbar(surf1, ax = ax1, shrink = 0.3, label = r'Absorption ($\frac{1}{m}$)',location = 'left')

#ax1.set_xlim(420,720)
#ax1.set_zlim(0, 1.2)
ax1.set_xlabel('Depth')
ax1.set_ylabel('Wavelength',labelpad = 20)
ax1.set_title(profile.time.min())
plt.savefig('/home/jovyan/acs_workshop/examples/gif_making/a_pro10_fig.png')

In [ ]:
optaa.profile

In [ ]:
np.unique(optaa.wavelength_a)

In [ ]:
#making gif
import imageio
import os

def gif_maker_profiles(ds,gif_name):
    filenames = []
    for pnum in np.unique(optaa.profile):
        #make plot
        profile = profiles[pnum]
        XA = np.array(profile.wavelength_number.values)
        YA = np.array(profile.depth.values).astype(np.float64)
        XA, YA = np.meshgrid(XA, YA)
        ZA = np.array(profile.apg_ts_s.values)
        XC = np.array(profile.wavelength_number.values)
        YC = np.array(profile.depth.values).astype(np.float64)
        XC, YC = np.meshgrid(XC, YC)
        ZC = np.array(profile.cpg_ts.values)

        fig = plt.figure(figsize = (11,5.5))
        grid = (1,1)
        ax1 = plt.subplot2grid(grid, (0,0), colspan = 1, rowspan = 1, projection = '3d')

        surf1 = ax1.plot_surface(YA, XA, ZA, cmap = 'jet', linewidth = 0, antialiased = True)

        fig.colorbar(surf1, ax = ax1, shrink = 0.3, label = r'Absorption ($\frac{1}{m}$)',location = 'left')

        ax1.set_ylim(0,85)
        #ax1.set_xticks(np.linspace(0,10, len(np.unique(optaa.wavelength_a.values)), np.unique(optaa.wavelength_a.values)))
        ax1.set_xlim(0,40)
        ax1.set_zlim(0, 1.65)
        ax1.set_zlabel('Absorption')
        ax1.set_ylabel('Depth')
        ax1.set_xlabel('Wavelength',labelpad = 20)
        ax1.set_title("Profile ;) {}".format(pnum))

        # create file name and append it to a list
        filename = 'profiles_animation{}.png'.format(str(pnum))
        filenames.append(filename)
        # save frame
        plt.savefig(filename,bbox_inches='tight')
        plt.close()

    # build gif
    with imageio.get_writer(gif_name, mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Remove files
    for filename in set(filenames):
        os.remove(filename)

In [ ]:
gif_maker_profiles(profiles,"check_this_out2.gif")

In [ ]:
fig = plt.figure(figsize = (11,5.5))
grid = (1,1)
ax1 = plt.subplot2grid(grid, (0,0), colspan = 1, rowspan = 1, projection = '3d')

surf1 = ax1.plot_surface(YA, XA, ZA, cmap = 'jet', linewidth = 0, antialiased = True)

fig.colorbar(surf1, ax = ax1, shrink = 0.3, label = r'Absorption ($\frac{1}{m}$)',location = 'left')

#ax1.set_xlim(420,720)
#ax1.set_zlim(0, 1.2)
ax1.set_xlabel('Depth')
ax1.set_ylabel('Wavelength',labelpad = 20)
ax1.set_title(profile.time.min())

#plt.savefig('/home/jovyan/acs_workshop/examples/a_pro10_fig.jpg')

In [ ]:
fig = plt.figure(figsize = (11,8.5))
grid = (1,1)
ax1 = plt.subplot2grid(grid, (0,0), colspan = 1, rowspan = 1, projection = '3d')

surf1 = ax1.plot_surface(YC, XC, ZC, cmap = 'jet', linewidth = 0, antialiased = True)

fig.colorbar(surf1, ax = ax1, shrink = 0.3, label = r'Absorption ($\frac{1}{m}$)',location = 'left')

#ax1.set_xlim(420,720)
#ax1.set_zlim(0, 1.2)
ax1.set_xlabel('Depth')
ax1.set_ylabel('Wavelength',labelpad = 20)
ax1.set_title(profile.time.min())

In [ ]:
fig, ax = plt.subplots(figsize = (9,1.5), sharex = True)

p1 = ax.plot(depth.time.transpose(), depth.ratio_cdom.transpose(), color = 'blue', label = 'Ratio NUV to 440',)
#p3 = ax[0].plot(depth.time.transpose(), depth.estimated_chlorophyll.transpose(), color = 'blue',label = 'L2')

handles, labels = ax.get_legend_handles_labels()
#handles = [handles[0],handles[-1]]
#labels = [labels[0],labels[-1]]
ax.legend(handles, labels, loc = 'upper right')
ax.set_ylabel('ratio CDOM')


In [ ]:
profile

In [ ]:
fig, ax = plt.subplots(figsize = (9,1.5), sharex = True)

p1 = ax.plot(depth.profile.transpose(), depth.sea_water_practical_salinity.transpose(), color = 'green', label = 'Line height chl',)
#p3 = ax[0].plot(depth.time.transpose(), depth.estimated_chlorophyll.transpose(), color = 'blue',label = 'L2')

handles, labels = ax.get_legend_handles_labels()
#handles = [handles[0],handles[-1]]
#labels = [labels[0],labels[-1]]
ax.legend(handles, labels, loc = 'upper right')
ax.set_ylabel('estimated chl a')